In [23]:
# Import the packages
import torch
import os
import argparse
import numpy as np
import os.path as osp
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from tqdm import tqdm
#from torcheval.metrics import R2Score # To be implemented
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear
from torch.nn import BatchNorm1d
import torch.optim as optim

from aux_func import*
from datset_process import *
from torch.utils.data import DataLoader

# Import the pretrained default model resnet18/resnet50/resnet101
from torchvision.models import resnet50
from MODEL.model import resnet12

In [24]:
# Define arg parser
seed=200
paser = argparse.ArgumentParser()
args = paser.parse_args("")
np.random.seed(200)
torch.manual_seed(seed)
device=input("Enter cuda or cpu for device type")
device = torch.device(device)
#'cuda' if torch.cuda.is_available() else
device

device(type='cpu')

In [25]:
# Take user inputs 
args.dataset='miniimagenet'
args.data_path='datasets/mini-imagenet/'
args.num_classes=64 # By default for miniimagenet
args.image_size=84

# FSL definitions
args.num_ways=5 # Number of classes per batch
args.k_shot=5 # number of Images per class
args.query=15 # Query set of the FSL
args.unlabel=50 # Number of unlabel samples per class 
args.steps=5 # Select how many unlabeled data for each class in one iteration.
args.threshold=0.2  # Since we have 5 classes in each support set. So if all the classes are equally probable then mininmum p=0.2

# set in semi-supervised few-shot learning
num_support = args.k_shot * args.num_ways
num_query = args.query * args.num_ways
num_unlabeled = args.unlabel * args.num_ways

# Training or testing definitions 
args.episodes=600

In [26]:
# Number of sets of unlabeled data
num_select = int(args.unlabel / args.steps)

In [27]:
# Import the resnet model and define the model to be used 
model=resnet50(num_classes=args.num_classes,pretrained=False)
#model=resnet12(args.num_classes)

# define last layer

'''
The input shape of last layer is found from the output shape of the resnet layer defined as 
'''
# in_channels=2048*3*3 out_channels=args.num_ways
last_layer=nn.Linear(2048,args.num_ways,bias=True)
print(model)
model.fc = nn.Flatten()
print(model)

c:\Users\kanad\Desktop\Github repos\ME_793_PROJECT\793_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\kanad\Desktop\Github repos\ME_793_PROJECT\793_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [28]:
#  Get the features from the resnet model

def get_features(model,input):
    '''
    The function first checks if the input batch size exceeds a desired batch size. If it does, the input batch is split into smaller batches of size 64, and the 
    ResNet model is called on each smaller batch using the model function with the return_feat=True argument to return the output features in addition to the classification results. 
    The output features are then detached from the computation graph, transferred to the CPU, and appended to a list embed. 
    Once all batches have been processed, the list of output features is concatenated using torch.cat to form a single tensor embed.
    If the input batch size is less than or equal to the desired batch size, the ResNet model is called once with the input batch using the model function with the return_feat=True argument to return the output features.
    Finally, the function checks if the shape of the output features embed matches the shape of the input batch, and returns the output features as a NumPy array using the numpy() method.
    '''
    batch_size = 32  # Use the desired batch size
    input = torch.tensor(input)
    
    # Check to prevent the input shape from exceeding the desired batch size
    if input.shape[0] > batch_size:
        embed = []
        i = 0
        while i <= input.shape[0]-1:
            embed.append(model(input[i:i+batch_size])) # Changed. Check
            i += batch_size
        embed = torch.cat(embed)
    else:
        embed = model(input) # Changed. Check. Removed return feat
    assert embed.shape[0] == input.shape[0] # Check if input shape = embed shape  as we will be working on input shape.
    return embed.detach().numpy()

In [29]:
def train_loop(model1, dataset, loss_fn, optimizer, inputs, targets):
    # Define forward function
    def forward_fn(data, label):
        logits = model1(data)
        print(logits)
        loss = loss_fn(logits, label) + entropy_loss(logits) # Combination of two loss functions used here. The entropy_loss act as regularizer
        # loss = loss_fn(logits, label)
        return loss, logits
    
    def train_step(data, label):
        optimizer.zero_grad()
        loss, logits = forward_fn(data, label)
        loss.backward()
        optimizer.step()
        return loss.item(), logits
    
    model.train()
    inputs = torch.tensor(inputs)
    targets = torch.tensor(targets).long() # Changed
    loss, logits = train_step(inputs, targets)
    return loss, logits

In [30]:
def get_preds(out):
    preds = torch.argmin(out, dim=0).item()
    return preds, preds

In [31]:
def get_preds_position_(unlabel_out, position, _postion, thres=0.001):
    length = len(position)
    r = []
    un_idx = []
    for idx in range(length):
        pos = position[idx]
        _pos = _postion[idx]
        _out = unlabel_out[idx][pos]
        out = F.softmax(_out,dim=0)  # Check if dim=0 or 1 # Correct
        #print(out)
        if len(pos)==1:
            un_idx.append(idx)
            continue
        conf =  torch.argmin(out).item()
        conf = out.view(-1)[conf].item()
        #print(conf)
        if conf>thres:
            un_idx.append(idx)
            if len(_pos)==0:
                r.append(torch.argmin(out,dim=1).item().asnumpy())  # check if asnumpy works here or not
            else:
                r.append(_pos[-1])
            continue
        t, _ = get_preds(out)
        a = pos[t]
        _postion[idx].append(a)
        position[idx].remove(a)
        #print(position)
        r.append(a)
    return np.asarray(r), un_idx,_postion,position  # Changed here

In [32]:
# Define the dataset and the respective loaders
args.train_episodes=10
dataset = DataSet( args.image_size, 'train',args.data_path)
sampler = EpisodeSampler(dataset.label, args.train_episodes,args.num_ways, args.k_shot, args.query, args.unlabel)
trainloader = DataLoader(dataset, batch_sampler=sampler,shuffle=False, num_workers=4, pin_memory=True)


In [33]:
# Start the training process 

# in the below code I am not using query set which should be concated with unlabeled data.

for data in tqdm(trainloader):

        # create different sets of data from the train loader
        data = data.cpu()
        targets = torch.arange(args.num_ways).repeat(args.k_shot+args.query+args.unlabel).long()

        #print(data,targets)
    
        support_data = data[:num_support]
        query_data = data[num_support:num_support+num_query]
        unlabel_data = data[num_support+num_query:]

        support_inputs = normalize(get_features(model, support_data))  # get feature embeddings for 
        support_targets = targets[:num_support].cpu().numpy()

        print(support_inputs.shape,support_targets)

        query_inputs = normalize(get_features(model, query_data))
        query_targets = targets[num_support:num_support+num_query].cpu().numpy()

        #print(query_inputs.shape,query_targets.shape)

        unlabel_inputs = normalize(get_features(model, unlabel_data))
        unlabel_targets = targets[num_support+num_query:].cpu().numpy()

        # The classifier has already been decided as linear classifier with a single dense layer and the output dimension=5

        ori_index = [x for x in range(250)]  # Store the index position of 250 images
        _POSITION = [[] for _ in range(250)] # Create a 2D list to store the list of 5 classes in passed along with the image batch.
        POSITION = [[0, 1, 2, 3, 4] for _ in range(250)] # [0,1,2,3,4] was chosen for encoding the 5 classes
        
        # Define the loss criterion and the SGD optimizer used here for initial training of model.
        criterion = nn.CrossEntropyLoss(reduction='mean')
        #print(last_layer.parameters())
        optimizer = torch.optim.SGD(last_layer.parameters(), lr = 1e-3, momentum=0.9, weight_decay=5e-4)  # weight decay is for L2 regularization.

        # Begin initial training

        print('\n********************************************  Initial training the model')
        for epoch in range(100):
              loss,_=train_loop(last_layer, None, criterion, optimizer, support_inputs, support_targets)
              #print(f"Train_Epoch: {epoch}  Train_Loss: {loss}")
        
        # Start of code using complimentary labels
        i=1
        while(True):
            print('\n********************************************  Training with complimentatry labels')
            select_idx=[]
            nl_pred=[]
            unselect_idx=[]
            unlabel_out = last_layer(torch.tensor(unlabel_inputs))
            nl_pred, unselect_idx,_POSITION,POSITION = get_preds_position_(unlabel_out, POSITION, _POSITION, args.threshold)  # Changed
            #print(unselect_idx)
            print(_POSITION)
            select_idx = [x for x in ori_index if x not in unselect_idx]
            _unlabel_embeddings = unlabel_inputs[select_idx]
            #_unlabel_t = unlabel_targets[select_idx]
            nl_pred = nl_pred[select_idx]
            print(nl_pred,i)
            if(len(nl_pred)==0):
                  break
            optimizer_NL = torch.optim.SGD(last_layer.parameters(), lr = 1e-3, momentum=0.9, weight_decay=5e-4)
            for epoch in range(20):
                loss = train_loop(last_layer, None, NL_loss, optimizer_NL, unlabel_inputs, nl_pred)
                #print(f"Epoch: {epoch}  Loss: {loss}")
            
            # Break condition no negative label found below threshold condition
            i=i+1
            
        print(_POSITION) # Printing NULL . Check
        class_num = [0 for _ in range(5)]
        pseudo_label = []
        index_pl = []
        for idx in range(len(POSITION)):
            item = POSITION[idx]
            if len(item) == 1:
                lab = item[0]
                pseudo_label.append(item[-1])
                class_num[lab] += 1
                index_pl.append(idx)
        class_num = [item + 8 for item in class_num]
        max_ = max(class_num) * 1.0
        pseudo_label = np.asarray(pseudo_label)
        t1_ = unlabel_inputs[index_pl]
        t2_ = torch.tensor(pseudo_label, dtype=torch.int64)
        print(t2_)
        for epoch in range(100):
            loss=train_loop(last_layer, None, criterion, optimizer, t1_, t2_)
            print(f"Epoch: {epoch}  Loss: {loss}")   
        



  0%|          | 0/10 [00:00<?, ?it/s]C:\Users\kanad\AppData\Local\Temp\ipykernel_25876\2955394716.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(input)


(25, 2048) [0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4]


C:\Users\kanad\AppData\Local\Temp\ipykernel_25876\2955394716.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(input)
C:\Users\kanad\AppData\Local\Temp\ipykernel_25876\2955394716.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(input)



********************************************  Initial training the model
tensor([[ 0.0286,  0.0014,  0.0188,  0.0103, -0.0089],
        [ 0.0279, -0.0085,  0.0078,  0.0095, -0.0074],
        [ 0.0258, -0.0040,  0.0050,  0.0038, -0.0162],
        [ 0.0170,  0.0002,  0.0078,  0.0039, -0.0125],
        [ 0.0206, -0.0064,  0.0136,  0.0102, -0.0140],
        [ 0.0244, -0.0023,  0.0105,  0.0019, -0.0159],
        [ 0.0197, -0.0039,  0.0088,  0.0007, -0.0137],
        [ 0.0224, -0.0007,  0.0118,  0.0118, -0.0099],
        [ 0.0203, -0.0035,  0.0027,  0.0039, -0.0228],
        [ 0.0187,  0.0017,  0.0112,  0.0068, -0.0177],
        [ 0.0179, -0.0080,  0.0130,  0.0105, -0.0116],
        [ 0.0178, -0.0007,  0.0039,  0.0145, -0.0038],
        [ 0.0199,  0.0011,  0.0069,  0.0128, -0.0145],
        [ 0.0251,  0.0016,  0.0111,  0.0123, -0.0050],
        [ 0.0172,  0.0023,  0.0036,  0.0095, -0.0130],
        [ 0.0196,  0.0054,  0.0134,  0.0111, -0.0100],
        [ 0.0200, -0.0018,  0.0011,  0.0112, -

C:\Users\kanad\AppData\Local\Temp\ipykernel_25876\3623213113.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets).long() # Changed
 10%|█         | 1/10 [00:13<02:02, 13.56s/it]C:\Users\kanad\AppData\Local\Temp\ipykernel_25876\2955394716.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(input)


...

C:\Users\kanad\AppData\Local\Temp\ipykernel_25876\2955394716.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(input)
C:\Users\kanad\AppData\Local\Temp\ipykernel_25876\2955394716.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(input)



********************************************  Initial training the model
tensor([[ 0.0855,  0.0653,  0.1119,  0.0896, -0.3417],
        [ 0.1094,  0.0648,  0.0838,  0.0974, -0.3379],
        [ 0.0886,  0.0686,  0.1182,  0.0878, -0.3360],
        [ 0.0825,  0.0589,  0.1154,  0.0859, -0.3353],
        [ 0.1125,  0.0724,  0.0804,  0.1059, -0.3364],
        [ 0.1020,  0.0730,  0.1006,  0.1058, -0.3381],
        [ 0.1009,  0.0595,  0.1017,  0.0991, -0.3431],
        [ 0.0927,  0.0723,  0.1127,  0.0911, -0.3408],
        [ 0.0951,  0.0725,  0.1040,  0.0972, -0.3370],
        [ 0.0917,  0.0691,  0.1072,  0.0912, -0.3384],
        [ 0.0948,  0.0742,  0.0966,  0.0973, -0.3336],
        [ 0.1029,  0.0761,  0.0868,  0.0899, -0.3397],
        [ 0.0864,  0.0642,  0.1172,  0.0934, -0.3298],
        [ 0.0875,  0.0684,  0.1128,  0.0918, -0.3359],
        [ 0.0956,  0.0639,  0.1084,  0.0969, -0.3386],
        [ 0.0876,  0.0731,  0.1064,  0.0930, -0.3359],
        [ 0.1069,  0.0686,  0.0914,  0.0926, -

C:\Users\kanad\AppData\Local\Temp\ipykernel_25876\3623213113.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets).long() # Changed
100%|██████████| 10/10 [01:10<00:00,  5.94s/it]

tensor([], size=(0, 5), grad_fn=<AddmmBackward0>)
Epoch: 12  Loss: (nan, tensor([], size=(0, 5), grad_fn=<AddmmBackward0>))
tensor([], size=(0, 5), grad_fn=<AddmmBackward0>)
Epoch: 13  Loss: (nan, tensor([], size=(0, 5), grad_fn=<AddmmBackward0>))
tensor([], size=(0, 5), grad_fn=<AddmmBackward0>)
Epoch: 14  Loss: (nan, tensor([], size=(0, 5), grad_fn=<AddmmBackward0>))
tensor([], size=(0, 5), grad_fn=<AddmmBackward0>)
Epoch: 15  Loss: (nan, tensor([], size=(0, 5), grad_fn=<AddmmBackward0>))
tensor([], size=(0, 5), grad_fn=<AddmmBackward0>)
Epoch: 16  Loss: (nan, tensor([], size=(0, 5), grad_fn=<AddmmBackward0>))
tensor([], size=(0, 5), grad_fn=<AddmmBackward0>)
Epoch: 17  Loss: (nan, tensor([], size=(0, 5), grad_fn=<AddmmBackward0>))
tensor([], size=(0, 5), grad_fn=<AddmmBackward0>)
Epoch: 18  Loss: (nan, tensor([], size=(0, 5), grad_fn=<AddmmBackward0>))
tensor([], size=(0, 5), grad_fn=<AddmmBackward0>)
Epoch: 19  Loss: (nan, tensor([], size=(0, 5), grad_fn=<AddmmBackward0>))
tensor([

100%|██████████| 10/10 [01:10<00:00,  7.08s/it]
